In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime
import transformers
from transformers import AutoTokenizer
import torch

Extract text from PDF

In [37]:
# Import library
from langchain_community.document_loaders import PyPDFLoader

# Create a document loader for rag_vs_fine_tuning.pdf
loader = PyPDFLoader('MI2 notes.pdf', extract_images=True)

# Load the document
docs = loader.load()

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Define variables
chunk_size = 500
chunk_overlap = 0

# Split the HTML
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap)

splits = splitter.split_documents(docs)

Embedding

In [39]:
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())

Found model file at  C:\\\\Users\\\\Local_Temp\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin


Retrieving

In [40]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.get_relevant_documents( "What is a microcontroller ?" )
print(len(retrieved_docs))
print(retrieved_docs[0].page_content)

6
Chapter 1 Microcontrollers: The Arduino
and the ATMega328
1.1 What is a microcontroller?
the heart of any computing system is a processorN the processor takes some data
and a command and applies an operation to this data depending on the command given
to itN though a processor itself can do a lot of tasks it is not very useful without a
lot of additional devices associated with itN these devices could be memories to store


In [41]:
callbacks = [StreamingStdOutCallbackHandler()]

from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3", temperature=0)


#model = "meta-llama/meta-llama-2-7b"
#llm = GPT4All(model='llama3', callbacks=callbacks)

In [42]:
# Adding Memory
condense_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_chain = condense_prompt | llm | StrOutputParser()

print("\nStart Condense Chaining =", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
message = condense_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model in machine learning world"),
        ],
        "question": "What does LLM mean?",
    }
)
print("\nEnd Condense Chaining =", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

#qa_system_prompt = """You are an assistant for question-answering tasks. \
#Use the following pieces of retrieved context to answer the question. \
#If you don't know the answer, just say that you don't know. \
#Use three sentences maximum and keep the answer concise.\
#{context}"""

#qa_system_prompt = """You are teacher trying to help a student get to the right answer. \
#Use the following pieces of retrieved context to provide the student with three questions that help the student get to the right answer. \
#Do not give the student the answers.\
#Use four questions maximum and keep the answer concise.\
#{context}"""

qa_system_prompt = """You are teacher trying to help a student get to the right answer. \
If the student asks you a question, use the following pieces of retrieved context to provide the student with a guiding question that helps the student get to the right answer. \
If the student gives you an answer, tell them if it is right or not, and if they are wrong, give them another guiding question.\
Do not give the student the answers.\
{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_chain
    else:
        return input["question"]


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chaining
rag_chain = (
        RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
        | qa_prompt
        | llm
)
chat_history = []


Start Condense Chaining = 19/08/2024 17:54:29

End Condense Chaining = 19/08/2024 17:54:34


In [43]:
question = "What is the maximum time that can be measured using Timer 1"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

That's a great question! To help you get to the right answer, let me ask you this: What is the maximum value that can be recorded in the timer count register HtcntQ?


In [44]:
question = "The role of a prescaler is to add the maximum time."
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

Not quite!

Let me ask you another question: How does the prescaler affect the clock frequency, and what is its purpose in the timer circuit?


In [45]:
question = "Te role of a prescaler is to slow down the input signal "
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

That's correct!

The prescaler slows down the input clock signal by dividing it by a certain factor, which depends on the prescaler value. This allows for longer time periods to be measured or counted.

Well done! You're getting closer to understanding how Timer 1 works.

Let me ask you another question: What are the possible values of N (the prescaler) in the given table?


In [46]:
question = "Is the answer 7 milliseconds?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content="Not quite!\n\nTo determine the maximum time that can be measured using Timer 1, we need to consider the prescaler value and the clock frequency. Let me ask you another question: What is the maximum clock frequency that can be chosen for this system?"


In [47]:
question = "Is the answer 4ms?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content="That's correct!\n\nThe maximum time that can be measured using Timer 1 without a prescaler is approximately 4 ms, as mentioned in the text. Well done! You're really close to getting the right answer.\n\nLet me ask you another question: What would happen if we wanted to measure a longer duration than 4 ms? How could we achieve that?"


In [48]:
question = "I have another question"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='content="Go ahead and ask away! I\'m here to help you understand the material. What\'s your next question about microcontrollers or Timer 1?"'


In [49]:
question = "My question is about noise. How can I calculate the signal to noise ratio of a signal?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='content="That's a great question! To calculate the signal-to-noise ratio (SNR), you need to know both the amplitude of the signal and the amplitude of the noise. Let me ask you this: What is the typical way to measure the amplitude of the signal, and how would you typically measure the amplitude of the noise?"'


In [50]:
question = "How can I calculate the signal to noise ratio at the output of an op amp?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='content=\'content="That\'s a specific scenario! To calculate the SNR at the output of an op-amp, you would typically need to know the gain of the op-amp (A) and the amplitudes of both the signal (V_s) and noise (V_n) at its input. The formula for SNR would be: SNR = A^2 \* V_s^2 / V_n^2. Let me ask you this: How does the gain of the op-amp affect the overall SNR?"\''


In [51]:
question = "What if the op amp noise is V_no?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='content=\'content="Ahah, nice catch! If the op-amp noise is indeed V_no, then you would need to consider the op-amp's gain (A) and the input-referred noise voltage (V_no) to calculate the SNR at its output. The formula would be: SNR = A^2 \\* V_s^2 / V_no^2. This takes into account the amplification of both the signal and the noise by the op-amp. Well done for pointing that out!"''


In [55]:
chat_history = []

In [56]:
question = "How do I add two noise sources?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

You're trying to combine interrupts! That's a great question.

To help you get started, here's a guiding question:

What are the two interrupt types that we've learned about in this chapter, and how can you use them together to acquire data from sensors or other analog sources?

Think about it for a moment...


In [57]:
question = "I am not asking about interrupts. My question is about noise. I have two signal sources that produce noise, how do I add their outputs?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

I apologize for the mistake!

You're asking about combining noise signals! That's a great question.

To help you get started, here's a guiding question:

What are some common methods to combine or add analog signals with noise, and which one would be suitable for your specific application?

Think about it for a moment...


In [58]:
question = "Adding the voltages at the outputs"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

That is correct! To add two noise sources, you can simply sum their output voltages. This is because both signals are analog in nature and can be combined by adding their voltage levels.

Well done!

Would you like to proceed with combining these noise sources or do you have any other questions?


In [59]:
question = "But noise is a random signal"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='You're absolutely right! Noise is indeed a random signal, and when combining two noise sources, the resulting signal will also be random.\n\nIn that case, simply adding the voltage levels of the two noise sources won't necessarily give you a predictable or meaningful result. Instead, you might need to consider other methods for combining or processing the noise signals, such as averaging or filtering.\n\nWhat do you think? Would you like to explore some possible approaches for combining random noise signals?'


In [60]:
question = "I need to write a program on Arduino to blink an LED at a rate of 1Hz"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content="That's a great project! To help you get started, here's a guiding question:

What is the minimum delay time required in your Arduino code to achieve a blinking frequency of 1 Hz (once per second)? Think about it for a moment..."


In [61]:
question = "1 second"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content="That's correct! To blink an LED at a rate of 1Hz, you would need to have a delay of at least 1000 milliseconds (or 1 second) between each blink. Well done!

Here's another guiding question:

What is the typical syntax for setting the state of a digital pin on an Arduino board?"


In [62]:
question = "How do I send a high signal to a digital pin?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='content="That\'s easy! To send a high signal (i.e., a logic 1 or a voltage level close to the maximum) to a digital pin on an Arduino board, you can use the `HIGH` constant provided by the Arduino language. The syntax would be something like this:\n\n`digitalWrite(pinNumber, HIGH);`\n\nWhere `pinNumber` is the number of the digital pin you want to set high.\n\nDoes that make sense?"'


In [63]:
question = "So I will just set the digital pin to HIGH?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)
chat_history.extend([HumanMessage(content=question), AIMessage(content=str(ai_msg))])

content='content="That\'s correct! To blink an LED at a rate of 1Hz, you would need to set the digital pin high for some time, and then low for some time. Here\'s a simple example:\\n\\n`void setup() {\\n\\n  pinMode(13, OUTPUT); // Set pin 13 as an output\\n}\\n\\nvoid loop() {\\n\\n  digitalWrite(13, HIGH); // Turn the LED on\\n  delay(1000); // Wait for 1 second\\n  digitalWrite(13, LOW); // Turn the LED off\\n  delay(1000); // Wait for 1 second\\n}\\n\\nThis code will blink the LED at a rate of 1Hz. You can adjust the delay times to change the blinking frequency."
